# 0. Load dataset

In [1]:
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')
directory = 'dataset/' # local directory
directory = '/kaggle/input/jules-varne/dataset/' # kaggle directory

df = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Read the CSV file into a DataFrame
        currFile = pd.read_csv(os.path.join(directory, filename))
        # Concatenate the DataFrame to the existing one
        df = pd.concat([df, currFile], ignore_index=True)

# 1. Dataset Preprocessing

- Removing null data
- Converting string data to int
- Removing invalid data

In [2]:
# Remove columns

columns = [
    "RecordingTime [ms]", # Done - no null values actually there
    "Time of Day [h:m:s:ms]", # 
    "Participant", # Done: Remove Missing Value, Remove unusables, null, undefined and such values, 
    "Pupil Diameter Right [mm]", # Code is there, just put it together.
    "Pupil Diameter Left [mm]", # Code is there, just put it together.
    "Point of Regard Right X [px]", # ??
    "Point of Regard Right Y [px]", # ??
    "Point of Regard Left X [px]", # ??
    "Point of Regard Left Y [px]", # ??
    "Category Right", # Done - Encoding, Missing Value
    "Category Left", # DONE - Encoding, Missing Value
]


df_0_columns_separated = df[columns]

In [3]:
import matplotlib.pyplot as plt

df = df_0_columns_separated 
df["RecordingTime [ms]"].describe()

is_null_in_column = df['RecordingTime [ms]'].isnull().any()
is_null_in_column # No null values then!

df_1_rec_time_processed = df

In [4]:


import matplotlib.pyplot as plt

df = df_1_rec_time_processed
df["Time of Day [h:m:s:ms]"].describe()

is_null_in_column = df['Time of Day [h:m:s:ms]'].isnull().any()
is_null_in_column # same, no null

df_2_time_of_day = df

In [5]:
# print("Before: ")
# print(df["Participant"].unique())

# Issue: some are in string type, some are in number type

df = df_2_time_of_day
df = df[(df['Participant'] != 'Unidentified(Neg)') & (df['Participant'] != 'Unidentified(Pos)')]
df["Participant"] = pd.to_numeric(df["Participant"], errors='coerce')

# print("After: ")
# print(df_processed_test["Participant"].unique())

df_3_participant = df

In [6]:
df = df_3_participant


column_name = "Pupil Diameter Right [mm]"

df[column_name] = df[column_name].astype(str) 
df = df[pd.to_numeric(df[column_name], errors='coerce').notnull()]
df[column_name] = pd.to_numeric(df[column_name], errors='coerce') 
df = df[df[column_name] != 0] 

column_name = "Pupil Diameter Left [mm]"

df[column_name] = df[column_name].astype(str)
df = df[pd.to_numeric(df[column_name], errors='coerce').notnull()]  
df[column_name] = pd.to_numeric(df[column_name], errors='coerce') 
df = df[df[column_name] != 0] 


df_5_pupil_both = df

In [7]:
point_of_regard_idx = [
    "Point of Regard Right X [px]",
    "Point of Regard Right Y [px]",
    "Point of Regard Left X [px]",
    "Point of Regard Left Y [px]",
]


for point in point_of_regard_idx:
    # drop null
    df.dropna(subset=[point], inplace=True)
    df = df.drop(df[df[point] == '-'].index)
    a = pd.to_numeric(df[point])

df_9_point_of_regard = df_5_pupil_both

In [8]:
# Processing - Category Left, Category Right

stays = [
    "Fixation",
    "Saccade",
    "Blink"
]

goes = [
    "Separator",
    "-",
    "Left Click",
    # Null
]

# 1. Remove missing data
# 2. Convert to numeric values??

df = df_9_point_of_regard

df = df[df['Category Left'].isin(stays)]
df = df[df['Category Right'].isin(stays)]

df = pd.get_dummies(df, columns=['Category Left'], prefix=['Category Left'])
df = pd.get_dummies(df, columns=['Category Right'], prefix=['Category Right'])
df.head()
# df.had()

df_final = df
df = df_final

In [9]:
# mapping subId to ASD / TD 
metadata_dir = "./Metadata_Participants.csv" #local dir
metadata_dir = "/kaggle/input/junes-verne-metadata/Metadata_Participants.csv" #kaggle dir

metadata = pd.read_csv(metadata_dir)
subjectClass = dict()
for idx, row in metadata.iterrows():
    subject = (row["ParticipantID"])
    category = (row["Class"])
    subjectClass[subject] = 1 if category == 'ASD' else 0

def getClass(subjectId):
    return subjectClass[subjectId]

# for index, row in df.iterrows():
#     subId = row["Participant"]
#     print(subId)

In [10]:
y = []
# Assigning class
for index, row in df.iterrows():
    y.append(getClass(row["Participant"]))

y
X = df

In [11]:
# df.to_csv('dataset_preprocessed.csv', index=False)

After grouping by sessions is done, we can do this.

- Consider each individual session as a singular data unit.
- From each units of data, compute all the feautures, and feed it into the classifier for proper patterns recognition
- Goal is to find as many features, correlated to ASD vs TD as possible.
- Hypothesis is there should be subtle patterns in eye movement speeds, fixation duration, and other similar charecteristics
- Eye pupil diameters reveal one's interest in a certain object / event / topic. Based on this, we can calculate how long a subject is able to hold his/her interest, and possibility is, that people with ASD will have different duration they can hold interests, ideally more on objects, less on people, and maybe similar to ADHD, there is some other patterns that otherwise would've been overlooked, in the topic of interest. Possible patterns:
    - How long can a subject hold interest
    - How often does he/she change it
    - Fatigue? How long till he loses interest, in a given session (is this correlated?)
    - Saccade movement, and pupil diameter, is there any correlation / patterns
- Eye movement is typically faster in ASD (ref. - )
- Ability to focus on an object right after fast switching of gaze, is slower among ASD (ref. - )
- Eye movements should be far less among TD than ASD, ASD people's gaze movement should be faster, little erratic, and possibly a bit jittery. Unable to focus.
- 

# 2. Feature Engineering
<!-- Features to compute:
- [x] Gaze Speed
- [x] Fixation Duration

As is:

- [x] Pupil Diameter
- [x] Category left, right
- [ ] 

Not sure if necessary:
- [x] Point of regards
- [x] Time of day (Can be used for separating sessions)
- [x] Recording Time (can be used for separating sessions)

---

After:

- Distance between points
- Gaze Speed
- Fixation duration
- Eye movement type (Fixation, Saccade, Blink) - 6 Cols, after encoding
- Pupil Diameter (Left, Right)
- Changes / Difference in fixation duration between consequent points
- Changes / Difference in distance between consequent points -->


## Separating Different Sessions
Using ID to separate different sessions.


In [12]:
allSessions = []
y = []
currSession = pd.DataFrame()

prevId = -1
cnt = 0
for index, row in df.iterrows():
    if row["Participant"] != prevId:
        allSessions.append(currSession.T)
        y.append(row["Participant"])
        currSession = pd.DataFrame()
    prevId = row["Participant"]
    currSession = pd.concat([currSession, row], axis=1)
    
# allSessions = pd.DataFrame(allSessions)
# allSessions

So there are 568 Sessions. Not bad. Here's the output shape:



In [13]:
# allSessions

## Generating output data (expected categories, ASD vs TD)
ASD = 0

TD = 1

In [14]:
mx = 0
for row in allSessions:
    mx = max(len(row), mx)
    # print(len(row))
print(mx)

12055


### Setting output values (expected classes)

In [15]:
X = allSessions
X.pop(0)

y = []

# First separating the subject ids. Later I'll compute the class (ASD / TD)
for x in X:
    # y.append(x["Participant"].first)
    # print(x["Participant"].first)
    # print(x["Participant"].shape)
    # print(x["Participant"].shape)
    # print(x.shape)
    y.append(x.iloc[0]["Participant"])

# print(y)

In [16]:
for i in range(len(y)):
    y[i] = getClass(y[i])


In [17]:
# Delete first empty element from the allSession df
total_elements = 0
for session in allSessions:
    # print(session.shape)
    total_elements += session.shape[1]
print(total_elements)

8505


## Computing Features: Distance, Changes in distance.

In [18]:
import math 

def getDistanceBetweenPoints(row, prevRow):
    leftX = float(row["Point of Regard Left X [px]"])
    leftY = float(row["Point of Regard Left Y [px]"])
    rightX = float(row["Point of Regard Right X [px]"])
    rightY = float(row["Point of Regard Right Y [px]"])

    x1 = (leftX + rightX) / 2
    y1 = (leftY + rightY) / 2

    prevLeftX = float(prevRow["Point of Regard Left X [px]"])
    prevLeftY = float(prevRow["Point of Regard Left Y [px]"])
    prevRightX = float(prevRow["Point of Regard Right X [px]"])
    prevRightY = float(prevRow["Point of Regard Right Y [px]"])

    x2 = (prevLeftX + prevRightX) / 2
    y2 = (prevLeftY + prevRightY) / 2

    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    return distance


def getChangesInDistanceBetweenPoints(currDist, prevDist):
    return abs(currDist - prevDist)

def getGazeSpeed(dist, duration):
    return dist / duration

In [19]:
# feat1_dist = []
# feat2_dist_diffs = []
# # feat3_speed = []

# prevRow = None
# prevDist = 0
# for i, row in df.iterrows():
#     if prevRow is None:
#         prevRow = row
#     dist = getDistanceBetweenPoints(row, prevRow)
#     dist_diff = getChangesInDistanceBetweenPoints(dist, prevDist)
#     # speed = 

#     prevRow = row
#     prevDist = dist

#     feat1_dist.append(dist)
#     feat2_dist_diffs.append(dist_diff)

# print(feat1_dist)
# print(feat2_dist_diffs)

In [20]:
# # To DataFrame
# feat1_dist = pd.DataFrame(feat1_dist)
# feat2_dist_diffs = pd.DataFrame(feat2_dist_diffs)

# # Set column names / feature names
# feat1_dist.columns = ["Distance"]
# feat2_dist_diffs.columns = ["Distance Difference"]

In [21]:
# as_is = [
#     "Participant",
#     "Category Left_Blink",
#     "Category Left_Fixation",
#     "Category Left_Saccade",
#     "Category Right_Blink",
#     "Category Right_Fixation",
#     "Category Right_Saccade",
#     "Pupil Diameter Right [mm]",
#     "Pupil Diameter Left [mm]",
# ]

# df_as_is = df[as_is] # First taking the as is ones

# # Combine all the features into the single input variable. 
# result = pd.concat([df_as_is, feat1_dist, feat2_dist_diffs], axis=1, join='outer')
# # X = result

### Compute feature for each different session

In [22]:
# Compute the features
tempX = X
feat1 = []
feat2 = []

for i in range(len(tempX)):
    currX = tempX[i]

    feat1_dist = []
    feat2_dist_diffs = []
    # feat3_speed = []

    prevRow = None
    prevDist = 0
    for i, row in df.iterrows():
        if prevRow is None:
            prevRow = row
        dist = getDistanceBetweenPoints(row, prevRow)
        dist_diff = getChangesInDistanceBetweenPoints(dist, prevDist)
        # speed = 

        prevRow = row
        prevDist = dist

        feat1_dist.append(dist)
        feat2_dist_diffs.append(dist_diff)
    
    # TODO: add each feat as row to each feat of tempX[i]
    feat1_df = pd.DataFrame(feat1_dist)
    feat2_df = pd.DataFrame(feat2_dist_diffs)
    tempX[i] = pd.concat([tempX[i], feat1_df], axis=1)
    tempX[i] = pd.concat([tempX[i], feat2_df], axis=1)

    
    # print(feat1_dist)
    # print(feat2_dist_diffs)
    # feat1.append(feat1_dist)
    # feat2.append(feat2_dist_diffs)

feat1

KeyboardInterrupt: 

In [ ]:
# Remove Unnecessary Featuers
tempX = X
for i in range(len(tempX)):
    tempX[i] = tempX[i][as_is]

tempX[0]

# Training - RNN

In [ ]:
# result

In [ ]:
# for x in X:
#     print(x.shape)

# X[0]

In [ ]:
def pad_dataset(data, target_length):
    num_padding_rows = target_length - data.shape[0]
    padding = np.zeros((num_padding_rows, data.shape[1]))
    padded_data = np.concatenate((data, padding))
    return padded_data

max_shape = max([data.shape[0] for data in X])

padded_X = [pad_dataset(data, max_shape) for data in X]

In [ ]:
# for x in padded_X:
#     print(x.shape)